In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import json

In [2]:

def login(mail,password,browser):
    browser.get("https://twitter.com/login")
    sleep(3)
    element_mail = browser.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[2]/form/div/div[1]/label/div/div[2]/div/input')
    element_pass = browser.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[2]/form/div/div[2]/label/div/div[2]/div/input')
    element_mail.send_keys(mail)
    element_pass.send_keys(password)
    element_pass.send_keys(Keys.RETURN)
    
    
def get_tweet_info(card,browser):
    try:
        name = username = card.find_element_by_xpath('.//span').text
    except NoSuchElementException:
        return
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    return {
            "name":name,
            "handle":handle,
            "postdate":postdate,
            "text":text,
            "reply_cnt" :reply_cnt,
            "retweet_cnt":retweet_cnt,
            "like_cnt":like_cnt
            }

In [3]:
browser = webdriver.Chrome('chromedriver')
password,mail = json.load(open("pass.json"))['pass'],json.load(open("pass.json"))['mail']
login(mail,password,browser)
browser.get('https://twitter.com/explore')
sleep(3)
element_search = browser.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[1]/div[1]/div/div/div/div/div[1]/div[2]/div/div/div/form/div[1]/div/div/div[2]/input')
element_search.send_keys("#polynote")
element_search.send_keys(Keys.RETURN)
sleep(3)
browser.find_element_by_link_text('Latest').click()
sleep(3)

In [5]:
data=[]
tweet_ids=set()

last_position = browser.execute_script("return window.pageYOffset;")
scrolling = True

while(True):
    tweet_blocks = browser.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in tweet_blocks:
        tweet = get_tweet_info(card,browser)
        if tweet:
            tweet_identifier = tweet['name']+tweet['handle']+tweet['text']
            if tweet_identifier not in tweet_ids:
                tweet_ids.add(tweet_identifier)
                data.append(tweet)
    scroll_attempt = 0
    while True:
        # check scroll position
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = browser.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

KeyboardInterrupt: 

In [6]:
data

[{'name': 'Nick Vintila',
  'handle': '@semanticbeeng',
  'postdate': '2021-03-12T09:03:08.000Z',
  'text': '#Polynote - "polyglot notebook environment" supports #Scala and #Python and is built with #functionalProgramming libraries like cats and zio but also #jep (Java Embedded Python for #JVM #CPython interop in same address space).\n\nPolyglot #softwareEngineeringQuote Tweet\nFrançois Armand\n@fanf42\n · Dec 12, 2019\n#FunScala2019 after starting with cats, #polynote switched to #zio for the free cancellable everything... And they remained for the environment\nShow this thread',
  'reply_cnt': '',
  'retweet_cnt': '1',
  'like_cnt': ''},
 {'name': 'Scott Weitzner',
  'handle': '@scott_weitzner',
  'postdate': '2020-12-21T14:18:17.000Z',
  'text': 'https://github.com/scottweitzner/docker-polynote… Polynote dockerized. A docker image based on \n@NetflixEng\n Polynote, a great alternative to JupyterLab (especially for scala) #polynote #cloudnative #CloudComputing #Docker **scottweitzner